In [1]:
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import sys

N = 10000 # number of samples per file
D = 3072 # number of features (dimensions of each picture)
K = 10 # number of classes

In [4]:
def LoadBatch(filename):
	""" Copied from the dataset website """
	import pickle
	with open(filename, 'rb') as fo:
		dict = pickle.load(fo, encoding='bytes')
	return dict

def montage(W):
	""" Display the image for each label in W """
	fig, ax = plt.subplots(2,5)
	for i in range(2):
		for j in range(5):
			im  = W[i*5+j,:].reshape(32,32,3, order='F')
			sim = (im-np.min(im[:]))/(np.max(im[:])-np.min(im[:]))
			sim = sim.transpose(1,0,2)
			ax[i][j].imshow(sim, interpolation='nearest')
			ax[i][j].set_title("y="+str(5*i+j))
			ax[i][j].axis('off')
	plt.show()

def loader(filepath):

    dic = LoadBatch("Datasets\\" + filepath)
    data = dic[b'data'].reshape(-1,3,32,32) # shape = d x n = 3072 x 10000
    labels = np.array(dic[b'labels']).flatten() # shape = n = 1000

    y = np.zeros((N, K))
    for i in range(N):
        y[i][labels[i]] = 1

    return data, y, labels

def extended_loader():
    Data = []
    OneHot = []
    Labels = []
    for i in range(5):
       D, H, L = loader(f"data_batch_{i+1}")
       Data.append(D)
       OneHot.append(H)
       Labels.append(L)
    return {"Data" :np.vstack(Data), "OneHot": np.vstack(np.array(OneHot)),"Labels" : np.hstack(Labels)}

def val_split(data,val_per=0.02):
    n_val = val_per * data["Labels"].shape[0]
    split = int(data["Labels"].shape[0] - n_val)
    train = {"Data" :data["Data"][:, :split], "OneHot": data["OneHot"][:, :split],"Labels" : data["Labels"][:split]}
    val = {"Data" :data["Data"][:, split:], "OneHot": data["OneHot"][:, split:],"Labels" : data["Labels"][split:]}
    return train, val

def mean_std(train_X):
    mean = np.mean(train_X, axis=1, keepdims=True)
    std = np.std(train_X, axis=1, keepdims=True)
    return mean, std

def normalize(mean, std, X):
    norm_X = (X - mean) / std
    return norm_X

def summarize_diagnostics(history):
 # plot loss
    plt.subplot(211)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(history.history['accuracy'], color='blue', label='train')
    plt.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    plt.savefig(filename + '_plot.png')
    plt.close()

#### Loading the Dataset


In [5]:
#load and split
Data = extended_loader()

train_X, train_Y, train_y = Data.values() # Data, one hot labels, labels
test_X, test_Y, test_y = loader("test_batch")

print('Train: X=%s, Y=%s, y=%s' % (train_X.shape, train_Y.shape, train_y.shape))
print('Test: X=%s, Y=%s, y=%s' % (test_X.shape, test_Y.shape, test_y.shape))

Train: X=(50000, 3, 32, 32), Y=(50000, 10), y=(50000,)
Test: X=(10000, 3, 32, 32), Y=(10000, 10), y=(10000,)


In [6]:
train_X = torch.tensor(train_X)
test_X = torch.tensor(test_X)
train_Y = torch.tensor(train_Y) 
test_Y = torch.tensor(test_Y)
train_y = torch.tensor(train_y)
test_y = torch.tensor(test_y)

TypeError: only size-1 arrays can be converted to Python scalars

#### Normalizing

In [13]:
train_X_norm = train_X / 255.0
test_X_norm = test_X / 255.0

#### Putting data in a Loader

In [14]:
train_set = TensorDataset(train_X_norm, train_Y)
train_loader = DataLoader(train_set, batch_size= 64, shuffle=True )

test_set = TensorDataset(test_X_norm, test_Y)
test_loader = DataLoader(test_set, batch_size= test_X_norm.shape[0], shuffle=False )

#### Test output size


$\text{Output\_size} = \left( \frac{\text{Input\_size} - \text{Kernel\_size} + 2 \times \text{Padding}}{\text{Stride}} \right) + 1$

In [15]:
conv_layer = nn.Conv2d(3, 32, kernel_size=(3, 3), padding=1)

input_tensor = torch.randn(1, 3, 32, 32) # (batchsize, channels, height, width)

output_tensor = conv_layer(input_tensor)

print(output_tensor.shape)

torch.Size([1, 32, 32, 32])


#### One Block VGG

In [23]:
class Cifar10Model_1block(nn.Module):
    def __init__(self):
        super(Cifar10Model_1block,self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,3), padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=(3,3), padding=1)

        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(32*16*16, 128)
        self.fc2 = nn.Linear(128, 10)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.relu(self.conv1(x)) # out = 32,32,32 (channels, height, width)
        x = self.relu(self.conv2(x)) # out = 32,32,32
        x = self.pool(x) # out = 32,16,16
        
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x)) # out = 128
        x = self.fc2(x) # out = 10
        return self.softmax(x)
    


In [32]:
def fit(model, dataLoader, Loss, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(dataLoader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = Loss(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:
                print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0


def evaluate_model(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for input, label in testloader:
            outputs = model(input)
            _, predicted = torch.max(outputs, 1)
            _, labels = torch.max(label, 1)            
            total += label.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy on the test set: {100 * correct / total:.2f}%')


In [25]:
model = Cifar10Model_1block()
Loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
n_epochs = 10

In [26]:
fit(model, train_loader, Loss, optimizer, n_epochs)

[Epoch 1, Batch 100] loss: 2.302
[Epoch 1, Batch 200] loss: 2.302
[Epoch 1, Batch 300] loss: 2.302
[Epoch 1, Batch 400] loss: 2.301
[Epoch 1, Batch 500] loss: 2.301
[Epoch 1, Batch 600] loss: 2.300
[Epoch 1, Batch 700] loss: 2.299
[Epoch 2, Batch 100] loss: 2.297
[Epoch 2, Batch 200] loss: 2.294
[Epoch 2, Batch 300] loss: 2.294
[Epoch 2, Batch 400] loss: 2.292
[Epoch 2, Batch 500] loss: 2.290
[Epoch 2, Batch 600] loss: 2.288
[Epoch 2, Batch 700] loss: 2.284
[Epoch 3, Batch 100] loss: 2.272
[Epoch 3, Batch 200] loss: 2.267
[Epoch 3, Batch 300] loss: 2.262
[Epoch 3, Batch 400] loss: 2.251
[Epoch 3, Batch 500] loss: 2.241
[Epoch 3, Batch 600] loss: 2.231
[Epoch 3, Batch 700] loss: 2.223
[Epoch 4, Batch 100] loss: 2.210
[Epoch 4, Batch 200] loss: 2.198
[Epoch 4, Batch 300] loss: 2.195
[Epoch 4, Batch 400] loss: 2.185
[Epoch 4, Batch 500] loss: 2.185
[Epoch 4, Batch 600] loss: 2.182
[Epoch 4, Batch 700] loss: 2.173
[Epoch 5, Batch 100] loss: 2.170
[Epoch 5, Batch 200] loss: 2.158
[Epoch 5, 

In [34]:
evaluate_model(model, test_loader)

Accuracy on the test set: 10.00%


#### Three Block Full Baseline

In [ ]:
class Cifar10Model(nn.Module):
    def __init__(self):
        super(Cifar10Model,self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,3), padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=(3,3), padding=1)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=(3,3), padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=(3,3), padding=1)

        self.conv5 = nn.Conv2d(64, 128, kernel_size=(3,3), padding=1)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=(3,3), padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.relu(self.conv1(x)) # out = 32,32,32 (channels, height, width)
        x = self.relu(self.conv2(x)) # out = 32,32,32
        x = self.pool(x) # out = 32,16,16
        
        x = self.relu(self.conv3(x)) # out = 64,16,16
        x = self.relu(self.conv4(x)) # out = 64,16,16
        x = self.pool(x) # out = 64,8,8
        
        x = self.relu(self.conv5(x)) # out = 128,8,8
        x = self.relu(self.conv6(x)) # out = 128,8,8
        x = self.pool(x) # out = 128,4,4
        
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x)) # out = 128
        x = self.fc2(x) # out = 10
        return self.softmax(x)
    


Print model layers

In [ ]:
model = Cifar10Model()
model

In [ ]:
def fit(model, dataLoader, Loss, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(dataLoader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = Loss(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:
                print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0


def evaluate_model(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for input, label in testloader:
            outputs = model(input)
            _, predicted = torch.max(outputs, 1)
            _, labels = torch.max(label, 1)            
            total += label.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy on the test set: {100 * correct / total:.2f}%')


Inintialising model and it's params

In [ ]:
model = Cifar10Model()
Loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
n_epochs = 2

Training

In [ ]:
fit(model, train_loader, Loss, optimizer, n_epochs)

Evaluation

In [ ]:
evaluate_model(model, test_loader)

#### Baseline + Dropout

In [ ]:
class Cifar10Model(nn.Module):
    def __init__(self, dropout_rate=0.0, batch_norm=False):
        super().__init__()
        self.batch_norm = batch_norm
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn1 = nn.BatchNorm2d(32) if batch_norm else nn.Identity()
        self.dropout1 = nn.Dropout(dropout_rate)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn2 = nn.BatchNorm2d(64) if batch_norm else nn.Identity()
        self.dropout2 = nn.Dropout(dropout_rate)
        
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn3 = nn.BatchNorm2d(128) if batch_norm else nn.Identity()
        self.dropout3 = nn.Dropout(dropout_rate)
        
        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.bn4 = nn.BatchNorm1d(128) if batch_norm else nn.Identity()
        self.dropout4 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn1(self.conv2(x)))
        x = self.pool1(x)
        x = self.dropout1(x)
        
        x = self.relu(self.bn2(self.conv3(x)))
        x = self.relu(self.bn2(self.conv4(x)))
        x = self.pool2(x)
        x = self.dropout2(x)
        
        x = self.relu(self.bn3(self.conv5(x)))
        x = self.relu(self.bn3(self.conv6(x)))
        x = self.pool3(x)
        x = self.dropout3(x)
        
        x = x.view(x.size(0), -1)
        x = self.relu(self.bn4(self.fc1(x)))
        x = self.dropout4(x)
        x = self.fc2(x)
        return x

def create_optimizer(model, lr=0.001, momentum=0.9, weight_decay=0.0):
    return optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

def train_model(model, trainloader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(trainloader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:
                print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0

def evaluate_model(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for input, label in testloader:
            outputs = model(input)
            _, predicted = torch.max(outputs, 1)
            _, labels = torch.max(label, 1)            
            total += label.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

model = Cifar10Model(dropout_rate=0.5, batch_norm=False)

criterion = nn.CrossEntropyLoss()
optimizer = create_optimizer(model, lr=0.001, momentum=0.9, weight_decay=0)

train_model(model, train_loader, criterion, optimizer)
evaluate_model(model, test_loader)


#### Baseline + Weight Decay (L2)

In [ ]:
class Cifar10Model(nn.Module):
    def __init__(self, dropout_rate=0.0, batch_norm=False):
        super().__init__()
        self.batch_norm = batch_norm
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn1 = nn.BatchNorm2d(32) if batch_norm else nn.Identity()
        self.dropout1 = nn.Dropout(dropout_rate)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn2 = nn.BatchNorm2d(64) if batch_norm else nn.Identity()
        self.dropout2 = nn.Dropout(dropout_rate)
        
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn3 = nn.BatchNorm2d(128) if batch_norm else nn.Identity()
        self.dropout3 = nn.Dropout(dropout_rate)
        
        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.bn4 = nn.BatchNorm1d(128) if batch_norm else nn.Identity()
        self.dropout4 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn1(self.conv2(x)))
        x = self.pool1(x)
        x = self.dropout1(x)
        
        x = self.relu(self.bn2(self.conv3(x)))
        x = self.relu(self.bn2(self.conv4(x)))
        x = self.pool2(x)
        x = self.dropout2(x)
        
        x = self.relu(self.bn3(self.conv5(x)))
        x = self.relu(self.bn3(self.conv6(x)))
        x = self.pool3(x)
        x = self.dropout3(x)
        
        x = x.view(x.size(0), -1)
        x = self.relu(self.bn4(self.fc1(x)))
        x = self.dropout4(x)
        x = self.fc2(x)
        return x

def create_optimizer(model, lr=0.001, momentum=0.9, weight_decay=0.0):
    return optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

def train_model(model, trainloader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(trainloader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:
                print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0

def evaluate_model(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for input, label in testloader:
            outputs = model(input)
            _, predicted = torch.max(outputs, 1)
            _, labels = torch.max(label, 1)            
            total += label.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

model = Cifar10Model(dropout_rate=0, batch_norm=False)

criterion = nn.CrossEntropyLoss()
optimizer = create_optimizer(model, lr=0.001, momentum=0.9, weight_decay=0.0005)

train_model(model, train_loader, criterion, optimizer)
evaluate_model(model, test_loader)


#### Baseline + Data Augmentation( horizontal flipping + translation)

In [31]:
#will implement it's simple